# VacationPy

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [2]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import cartopy.crs as ccrs
import geoviews as gv
# Import API key
from api_keys import geoapify_key


In [3]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv('output_data/cities.csv')

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,kapa'a,22.0752,-159.3190,22.99,93,70,3.63,US,1701450014
1,1,isafjordur,66.0755,-23.1240,-0.98,69,75,3.60,IS,1701450015
2,2,uturoa,-16.7333,-151.4333,27.12,81,49,8.23,PF,1701450016
3,3,waitangi,-43.9535,-176.5597,10.46,62,87,5.00,NZ,1701450019
4,4,adamstown,-25.0660,-130.1015,21.49,73,8,4.12,PN,1701450026


### Step 1: Create a map that displays a point for every city in the city_data_df DataFrame. The size of the point should be the humidity in each city.

In [9]:
%%capture --no-display

# Configure the map plot
map_plot_city = city_data_df.hvplot.points("Lng","Lat", geo = True, tiles="OSM", \
                                        frame_width=700, frame_height=500, \
                                        size="Humidity", scale=0.5, color="City")

# Display the map
map_plot_city


:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the city_data_df DataFrame to find your ideal weather condition

In [10]:
# Narrow down cities that fit criteria and drop any results with null values
temp_condition = ((city_data_df["Max Temp"] > 20) & (city_data_df["Max Temp"] < 27))
wind_condition = (city_data_df["Wind Speed"] < 5)
#humid_condition = ((city_data_df["Humidity"] > 20) & (city_data_df["Humidity"] < 60))
#note - removing the above condition, since its resulting in a fewer data set.
cloud_condition = (city_data_df["Cloudiness"] < 10)

city_data_fil = city_data_df.loc[(temp_condition & wind_condition & cloud_condition ), :]

# Drop any rows with null values
city_final = city_data_fil.dropna()

# Display sample data
city_final


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
4,4,adamstown,-25.0660,-130.1015,21.49,73,8,4.12,PN,1701450026
12,12,as sulayyil,20.4607,45.5779,24.14,30,7,4.28,SA,1701450028
14,14,cabo san lucas,22.8909,-109.9124,24.73,54,0,1.79,MX,1701449765
43,43,ghat,24.9647,10.1728,22.67,11,0,2.15,LY,1701450043
83,83,francistown,-21.1700,27.5079,25.94,26,1,4.61,BW,1701450053
87,87,richards bay,-28.7830,32.0377,20.65,66,0,4.54,ZA,1701450053
96,96,fernandina beach,30.6697,-81.4626,26.68,74,0,4.63,US,1701449841
174,174,az zuwaytinah,30.9522,20.1202,21.06,47,0,1.44,LY,1701450071
220,220,queenstown,-31.8976,26.8753,26.47,15,1,2.36,ZA,1701450091
268,268,pisco,-13.7000,-76.2167,23.90,68,0,4.12,PE,1701450117


### Step 3: Create a new DataFrame called hotel_df.

In [11]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = city_final.copy()

hotel_df = hotel_df[["City","Country","Lat","Lng","Humidity"]]

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
4,adamstown,PN,-25.0660,-130.1015,73,
12,as sulayyil,SA,20.4607,45.5779,30,
14,cabo san lucas,MX,22.8909,-109.9124,54,
43,ghat,LY,24.9647,10.1728,11,
83,francistown,BW,-21.1700,27.5079,26,
87,richards bay,ZA,-28.7830,32.0377,66,
96,fernandina beach,US,30.6697,-81.4626,74,
174,az zuwaytinah,LY,30.9522,20.1202,47,
220,queenstown,ZA,-31.8976,26.8753,15,
268,pisco,PE,-13.7000,-76.2167,68,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
# Set parameters to search for a hotel
radius = 10000
limit = 5
apiKey = geoapify_key
params = {
    "limit":limit,
    "apiKey":geoapify_key 
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    longitude = hotel_df.loc[index, "Lng"]
    latitude = hotel_df.loc[index, "Lat"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f'circle:{longitude},{latitude},{radius}'
    params["bias"] = f'proximity:{longitude},{latitude}'
    params["categories"] = f'accommodation.hotel'
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
     # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df
    

Starting hotel search
adamstown - nearest hotel: No hotel found
as sulayyil - nearest hotel: No hotel found
cabo san lucas - nearest hotel: Comfort Rooms
ghat - nearest hotel: فندق تاسيلي
francistown - nearest hotel: City Inn
richards bay - nearest hotel: SUN1 Richards Bay
fernandina beach - nearest hotel: Florida House Inn
az zuwaytinah - nearest hotel: No hotel found
queenstown - nearest hotel: Queens Hotel
pisco - nearest hotel: La Portada
ismailia - nearest hotel: Mercure
banda - nearest hotel: #acnindiafy21
winburg - nearest hotel: No hotel found
sur - nearest hotel: Sur Hotel
callao - nearest hotel: Casa Ronald
salalah - nearest hotel: Muscat International Hotel
del rio - nearest hotel: La Quinta Inn by Wyndham Del Rio
ras al khaimah city - nearest hotel: Oasis Apartment Hotel


,City,Country,Lat,Lng,Humidity,Hotel Name
4,adamstown,PN,-25.0660,-130.1015,73,No hotel found
12,as sulayyil,SA,20.4607,45.5779,30,No hotel found
14,cabo san lucas,MX,22.8909,-109.9124,54,Comfort Rooms
43,ghat,LY,24.9647,10.1728,11,فندق تاسيلي
83,francistown,BW,-21.1700,27.5079,26,City Inn
87,richards bay,ZA,-28.7830,32.0377,66,SUN1 Richards Bay
96,fernandina beach,US,30.6697,-81.4626,74,Florida House Inn
174,az zuwaytinah,LY,30.9522,20.1202,47,No hotel found
220,queenstown,ZA,-31.8976,26.8753,15,Queens Hotel
268,pisco,PE,-13.7000,-76.2167,68,La Portada


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [12]:
%%capture --no-display

# Configure the map plot
map_plot_hotel = hotel_df.hvplot.points("Lng","Lat", geo = True, tiles="OSM", \
                                        frame_width=700, frame_height=500, \
                                        size="Humidity", scale=0.5, color="City", \
                                        hover_cols = ["Hotel Name","Country"]
                                       )

# Display the map
map_plot_hotel

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)